# FFTHH November - Machine Learning Task

This month's FFTHH is going to be a basic machine-learning task, using some bread-and-butter tools.

<details>


<summary>What Are We Going to Do?</summary>

We're going to use some basic tools to do a basic machine learning task: reading digitized handwritten characters and classifying them as machine-readable digits.  

To do this, we'll use a well-known, publicly available dataset: the [MNIST database](https://en.wikipedia.org/wiki/MNIST_database) for handwritten digits handwritten digits:

Our task is to take the variable, noisy input from digits written by humans and reduce them to information easily understood by machines.  
![image](480px-MnistExamples.png "mnist-samples")

# &#x21D3;

[0, 1, 2, 3, 4, 5, etc... ]
</details>

<details>


<summary>Procedure</summary>

We'll be going through some of the steps that we do for machine-learning tasks, more generally. This means we'll be doing: 

- dataset building
- exploratory data analysis
- feature engineering
- model selection
- model training
- model validation
- hyperparameter tuning

We'll go through what each of these things means in more detail below.
</details>


<details>
<summary>What Tools are We Going to Use?</summary>

We'll be using the following libraries/frameworks:

- [Jupyter](https://jupyter.org/) notebooks running in an [Anaconda](https://www.anaconda.com/) environment for running our code in an interactive Python environment, with the needed data-science dependencies already pre-loaded
- [NumPy](https://numpy.org/) for manipulating matrices/arrays efficiently
- [Pandas](https://pandas.pydata.org/) for manipulating our datasets and doing feature engineering
- [Scikit-Learn](https://scikit-learn.org/stable/) for doing our main machine-learning heavy-lifting tasks
- [Matplotlib](https://matplotlib.org/) and [Graphviz](https://graphviz.org/) for doing visualization and charting

</details>

In [ ]:
import sklearn
from sklearn import model_selection as model_selection
from sklearn.tree import export_graphviz
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_halving_search_cv # noqa
from sklearn.model_selection import HalvingRandomSearchCV, cross_val_score
from scipy.stats import randint
from sklearn.metrics import plot_roc_curve

%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt

# Get a Dataset

In [ ]:
# this statement will use scikit-learn's built-in API for fetching datasets
# https://www.openml.org/search?type=data&sort=runs&id=554&status=active
dataset = fetch_openml("mnist_784")


In [ ]:
# first take a look around
dataset.data

In [ ]:
dataset.target

In [ ]:
# by default, sklearn returns datasets as tuples of numpy arrays. 
X, y = dataset.data, dataset.target

In [ ]:
# We'll be using Pandas for some of our feature engineering work, so we'll be transforming the datasets into Pandas dataframes.
# this will get our dataset as a single pandas dataframe. 
# this makes things like exploratory data analysis easy.

df_X, df_y = pd.DataFrame(X), pd.DataFrame(y)
df = df_X.join(df_y)


# Exploratory Data Analysis (EDA)

<details>
<summary>What is EDA?</summary>

The point of EDA is 
- To understand our dataset, 
- To inform our downstream tasks

Since we're doing an OCR Task, we'll want to know  what our dataset looks like, with that goal in mind. 

It helps us recognize problems and necessary data manipulation steps early.  We don't want to spend lots of money and time on model training or manual labelling tasks only to have to redo that work later. EDA can help us reduce the cycle time for our work.
</details>

<details>
<summary>What do we know already, before doing EDA?</summary>

First, it helps to consult the [documentation](https://www.openml.org/search?type=data&sort=runs&id=554&status=active). The documentation tells us 
- that we have a dataset with 784 features
- that it represents 28 x 28 pixel greyscale images, with the images centered in the frame, with pixel darkness ranging from 0 (white) to 255 (black).
</details>

<details>
<summary>What kinds of things do we learn by doing EDA?</summary>

for example:
- Variable Distributions
- Relationships between predictors
- missing values 
</details>

In [ ]:
# Figure out what the columns mean
# Visualization helps

digit_idx = 5000

# this gets us a row of pixel data
digit = df_X.iloc[digit_idx]

# reshape the data to be a matrix, and then chart it with matplotlib
digit_img = digit.to_numpy().reshape(28, 28)
plt.imshow(digit_img, cmap=matplotlib.cm.binary, interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
# see what the label for the features is
df_y.iloc[digit_idx]

### Now that we understand how our data are organized, let's perform some basic data hygiene tasks



In [ ]:
# get descriptive stats on subsets of pixels.
# pandas has a describe() method

df[[ 
    'pixel1', 'pixel2', 'pixel3', 
    'pixel391', 'pixel392', 'pixel393', 
    'pixel782', 'pixel783', 'pixel784']] \
.describe()


In [ ]:
df.hist(column=['pixel1', 'pixel2', 'pixel3', 'pixel391', 'pixel392', 'pixel393', 'pixel782', 'pixel783', 'pixel784'],
    bins=20)

# Feature Engineering

<details>

<summary>What has our minimal EDA shown us?</summary>

- The first pixels and the last pixels in the array are never (or almost never) filled in.
- By comparison, we see that the average number for the middle pixels, are, on average, darker. But still, they are far more likely to be blank.
- Even though theoretically, cells are bound between 0 and 255, the actual values are more varied
</details>

<details>
<summary>What should we do with this information?</summary>

Depending on which model we select, we may have to 'massage' these features so that they meet certain key assumptions of the model. 
"Massaging" our columns to make model performance better, is called "Feature Engineering"

</details>

### Missing Values

- Missing Values can be a show-stopper
- It looks like we didn't have any missing values this time. But what if we had? 
- *Imputation* means filling in missing values. There are a few different ways to do it

In [ ]:
# manual check for missing values

df.columns[df.isna().any()].tolist()

In [ ]:
# find a non-zero value, so we can artificially remove it
df.query('pixel392 > 0')['pixel392']


In [ ]:
# artificially remove a value

df.loc[22617, 'pixel392'] = np.nan
df.columns[df.isna().any()].tolist()

In [ ]:
# do imputation

# scalar imputation using forward- or back-fill *might* be a decent choice here, 
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.fillna.html#pandas.DataFrame.fillna

df['pixel392'].fillna(method='ffill', inplace=True)

# check what was imputed
print(df.loc[22617, 'pixel392'])

In [ ]:
# It just filled in zero because that's what the previous observation had for that pixel. Which, is really kind of arbitrary. 
# And, it's only using ONE piece of information to make the decision. 
# a better choice might be an imputer from SkLearn:
# https://scikit-learn.org/stable/modules/impute.html
df.loc[22617, 'pixel392'] = np.nan

from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')
imputer.fit(df.to_numpy())

In [ ]:
df_imputed = pd.DataFrame(imputer.transform(df))
df_imputed.columns = df.columns


In [ ]:
df_imputed[[ 
    'pixel1', 'pixel2', 'pixel3', 
    'pixel391', 'pixel392', 'pixel393', 
    'pixel782', 'pixel783', 'pixel784']] \
.describe()

In [ ]:
# list any missings - expect to be empty
print(df_imputed.columns[df_imputed.isna().any()].tolist())

# list the value for the imputed missing datum
print(df_imputed.loc[22617, 'pixel392'])

In [ ]:
# it's still off but it's better than the first one.
# there are better methods of imputation that the SimpleImputer, but for our purposes it's probably good enough. Let's move on.
df = df_imputed

### Was this imputation good? 

Maybe we could do better with an [Iterative Imputer](https://scikit-learn.org/stable/modules/generated/sklearn.impute.IterativeImputer.html#sklearn.impute.IterativeImputer).


### Scaling
<details>

<summary>What is Scaling?</summary>


Scaling means changing the value of each data point to be within a specific range. 
- Sometimes it is a linear transformation, like multiplying each data value in a column by a constant. 
- Sometimes it is not linear, like applying an exponential function or using a mathematical distribution to reassign a value.
</details>

<details>
<summary>Why scaling?</summary>


- Some models have a prerequisite that each column is bounded by values that are the same, or similar
</details>


<details>
<summary>What kinds of scaling are there?</summary>


- Min-max scaling
- Standard scaling
- other variations (e.g., Adaptive Scaling)
</details>

In [ ]:
# doing scaling on the features:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
scaler = MinMaxScaler()

# scaler fit_transform calls fit() and transform() all in one, for convenience
for x in df.columns:
    if x.startswith('pixel'):
        df[f'{x}_scaled'] = scaler.fit_transform(df[x].to_numpy().reshape(-1, 1))


In [ ]:
# verify that our scaling worked

df[[ 
    'pixel1_scaled', 'pixel2_scaled', 'pixel3_scaled', 
    'pixel391_scaled', 'pixel392_scaled', 'pixel393_scaled', 
    'pixel782_scaled', 'pixel783_scaled', 'pixel784_scaled']] \
.describe()


# Model Selection after Feature Engineering

<details>
<summary>What is Model Selection?</summary>

- It's really just selecting a model.
Realistically, Feature Engineering and Model Selection are often iterative processes. After Exploratory Data Analysis, we might try an initial round of feature engineering, followed by exploring some prototype models.  If those initial models don't prove to have good performance, we might try again by crafting new features or finding out how to improve the validity of the ones we have.  

For this exercise, though, we'll assume that we've done all of the feature engineering that we need to have done, and we can move on to selecting models.
</details>

<details>

<summary>How do we do it?</summary>


Short answer is that depends; it can be kind of hard.

Maybe the most important part of model selection is knowing enough about:
- The problem you're trying to solve
- The nature of your dataset

Assumptions (read 'requirements') that the dataset needs to meet, in order for the model to work.

In real practice, there's often some trial-and-error.  It is common to try multiple models and pick the one with the best results.
For example, for our problem (classifying numerals from digitized handwritten digits),  we might try ...: 
- Artificial neural nets
- Decision Trees
- Logistic regression

... We 
</details>


<details>
<summary>How do we make the decision?</summary>

1. The the model's appropriateness for the problem
- Categorization model vs Time-series model? 
- Regression Model
- Supervised/Unsupervised learning

2. The model's appropriateness for the data (does the model meet assumptions for interpretability of the results)
- Independence of observations from each other
- Independence of features from each other
- Normally distributed predictors (histogram of the predictor roughly follows Gaussian distribution)
- Homoskedasticity (variability in predictors needs to be unrelated to central tendency)

3. Operational constraints
- Will it be efficient to train?
- Is any special human effort involved (i.e., coding for training data)? 
- Will it be efficient to deploy and maintain over time?
</details>

<details>
<summary>What model did we pick?</summary>

For this exercise, we'll just pick a Random Forests Classifier

- It's easy to understand
- It has lax assumptions/requirements
- It shows us how ensembles work
</details>


# Training the Classifier

## Example Model: Random Forest Classifier (RFC)

<details>
<summary>What are Random Forests?</summary>


Random Forests are an Ensemble classifier made up of Decision Trees

</details>

<details>
<summary>So, then what are Decision Trees?</summary>

A Decision Trees is a classifier that works by branching conditional probabilities. Simply put, it is a process that takes input and tries to find the threshold points in different features (given prior choices) that can optimally sort the input into the target categories.

There is more than one possible target metric, but the default value that is used by scikit-learn for optimization is called the _Gini Impurity_.  Lower Gini impurity is an indicator of a the more accurate model, with a value of 0 representing a perfect model.

see: [gini impurity explained](https://towardsdatascience.com/gini-impurity-measure-dbd3878ead33)
</details>

<details>
<summary>Decision Trees Visualized</summary>

Example Decision Tree:
![image](tree_short.png "Example Decision Tree")


A Random Forest Classifier is simply a collection of Decision Trees. It is what is known as an 'ensemble' method, so called because the model is a collection of smaller models rather than being a single model. The collection of trees act as "voters" and the output class that has the most votes is the one chosen by the model.
</details>


In [ ]:
# Split the dataset by
# train versus test 
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
df_train, df_test = model_selection.train_test_split(df)

# split by 
# Features versus labels
# 1 pandas dataframe -> 4 numpy array
x_columns = list([c for c in df.columns if c.startswith('pixel') and c.endswith('scaled')])

def split_xy(df_whole, x_columnnames):
    x_df = df_whole[x_columnnames]
    y_df = df_whole['class']

    return x_df, y_df

df_train_x, df_train_y = split_xy(df_train, x_columns)
df_test_x, df_test_y = split_xy(df_test, x_columns)


In [ ]:
# looking at the shape of our data; it should be something that makes sense
print(df_train_x.shape)
print(df_train_y.shape) 
print(df_test_x.shape)
print(df_test_y.shape)


In [ ]:
# Random Forest Classifier

# in Scikit-learn, hyperparameters are passed into the classifier's constructor
rfclassifier = RandomForestClassifier(
    n_estimators=50,
    n_jobs=4)

rfclassifier.fit(X=df_train_x,y=df_train_y)




In [ ]:

# This RFC has hyperparameters that limit the depth of the tree and the number of samples per leaf node
stubbyrfc = RandomForestClassifier(
    n_estimators=10,
    max_depth=3,
    min_samples_leaf=10,
    n_jobs=4)

stubbyrfc.fit(X=df_train_x,y=df_train_y)


In [ ]:

# OPTIONAL STEP
# get a visualization of one of our trees.
# we can use Graphviz, which is an open source visualization tool 
# https://graphviz.org/download/

sample_tree = rfclassifier.estimators_[42]
export_graphviz(sample_tree,
    out_file='tree.dot',
    feature_names=x_columns,    
    rounded=True,
    proportion=False,
    precision=2,
    filled=True)



In [ ]:
stubby_sample = stubbyrfc.estimators_[4]
export_graphviz(stubby_sample,
    out_file='tree_short.dot',
    feature_names=x_columns,    
    rounded=True,
    proportion=False,
    precision=2,
    filled=True)

With the Graphviz tool we, can take a look at the topology of one of the decision trees that we built.

we need to do a little processing outside of this Jupyter notebook to invoke Graphviz to turn the `.dot` file into a png:

```
dot -Tpng .\tree.dot -o .\tree.png
```



One thing we notice right away is that it's BIG.  Here is what the whole thing looks like:

![image](tree.png "The Whole Tree")

That's hard to see. Here's a closer-up sample of the image:

![image](tree_sample.png "Sample of One Tree")


One thing that this image shows us is that we may have been lenient with the depth of our our trees, or how many branches we've allowed to be created.  

This is not necessarily bad. Models with lots of features and output classes might need to have more depth in order to build a well-performing set of trees.  However, a tree with unbounded or overly lenient hyperparameters that allow it to build out deeper and broader trees, this might be a cause of overfitting (if overfitting is found).  At any rate, We haven't yet established whether our model is indeed overfitting, so let's not get ahead of ourselves.  We will see below about what effect our model hyperparameters have on the shape of each individual tree, and on how the multiple trees behave together in the ensemble classifier.

# Evaluating Model Performance, Hyperparameter Tuning

Simple Evaluation
Cross-Validation
Tuning HPs
- Different HPs for different models.
- [Hyperparameters for a Random Forest Classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) include
    - Maximum depth
    - Minimum change in target metric before quitting
    - Minimum samples per node


In [ ]:
# Evaluating the model's initial prediction.
# Here we use our holdout sample to test whether we've done overfitting.
# If the model's performance on the holdout

pred_y = rfclassifier.predict(X=df_test_x)

conf_mat = confusion_matrix(y_true=df_test_y, y_pred=pred_y)
print(conf_mat)

In [ ]:
stb_pred_y = stubbyrfc.predict(X=df_test_x)

conf_mat = confusion_matrix(y_true=df_test_y, y_pred=stb_pred_y)
print(conf_mat)

### What we just did above with these two models is an example of **Hyperparameter Tuning**

<details>
<summary>What does the comparison between these two models tell us?</summary>

- The top one was better
- But why? we don't always know. In general, HP tuning provides a balance between:
    - Addressing overfitting vs
    - Providing sufficient complexity to handle the problem
</details>

<details>
<summary>Can we Automate Hyperparameter tuning?</summary>

[Yes!](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV)

Scikit-Learn comes with tools to explore different samplings of hyperparameters to help find optimal combinations of values.
</details>

In [ ]:
# Automated HP Tuning. This Takes about 11 minutes

# get an RFC with no HPs specified
# n_jobs and random_state don't affect the model characteristics
raw_rfc = RandomForestClassifier(n_jobs=4, random_state=42)
param_distributions = {
    'max_depth': [3, 10],
    'min_samples_split': randint(2, 11)
}

randomsearch_validator = HalvingRandomSearchCV(raw_rfc, 
    param_distributions, 
    random_state=42).fit(df_train_x, df_train_y)

randomsearch_validator.best_params_

Pasting the output of the above so that we don't have to run it for a long time:

![Image](HPTune_output.png "hp tuning")

## More Cross-Validation

Doing a single comparison with a holdout sample is good, but a good, generalizable model should be able to do that with more than just one split between estimation and test samples.  That is where cross-validation helps. It replicates the estimation/sample split with multiple subsamples.




In [ ]:
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html?highlight=cross_val_score#sklearn.model_selection.cross_val_score
cross_val_score(rfclassifier, 
    df_train_x, 
    df_train_y, 
    cv=3, 
    scoring="accuracy")

### Interpretation of Model Performance

So, we've gotten our metrics back about our model's performance. At first blush, it seems pretty good. But how does it translate to our business needs?  There are different metrics we can use to evaluate a model. Here are a few:

<details style="background-color:white;color:black;">
<summary>Accuracy</summary>

This is conceptually familiar, but what does it mean? 

![Image](accuracy.svg "accuracy")
</details>

<details style="background-color:white;color:black;">
<summary>Precision & Recall</summary>


How likely are we to correctly detect a true positive?

How likely are we to correctly reject a "false negative?"
![Image](recall.svg "recall")
</details>

<details>
<summary>ROC Plotting</summary>

How can we understand metrics like precision and recall as they pertain to our dataset?

Looking at plots of Receiver Operating Characteristics (ROC) can help

![Image](roccurves.png "example ROC curve")
source: By BOR at the English-language Wikipedia, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=10714489
</details>

<details>
<summary>What Does an ROC Plot Tell Us?</summary>

- Plots True Positives against False Positives
- Compares these 2 metrics at various model "thresholds"
</details>

In [ ]:
# Running this takes about 45 sec

import numpy as np
import matplotlib.pyplot as plt
from itertools import cycle

from sklearn import svm, datasets
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import roc_auc_score

# one-hot encoding for output vars
binary_y = label_binarize(df_train_y, classes=[0,1,2,3,4,5,6,7,8,9])
binary_y_test = label_binarize(df_test_y, classes=[0,1,2,3,4,5,6,7,8,9])
n_classes = binary_y.shape[1]

onevrest = OneVsRestClassifier(rfclassifier)
y_score = onevrest.fit(df_train_x, df_train_y).predict_proba(df_test_x)

# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(binary_y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure(figsize=(8,8), dpi=150)
lw=2
colors = cycle(["aqua", "darkorange", "cornflowerblue", "forestgreen", "firebrick", "coral", "rebeccapurple", "lightsalmon", "darkslategray", "cadetblue"])
for i, color in zip(range(n_classes), colors):
    print(f"i {i}, color {color}")    
    plt.plot(
        fpr[i],
        tpr[i],
        color=color,
        lw=lw,
        label="ROC curve of class {0} (area = {1:0.2f})".format(i, roc_auc[i]),
    )

plt.plot([0, 1], [0, 1], "k--", lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Some extension of Receiver operating characteristic to multiclass")
plt.legend(loc="lower right")
plt.show()

Using the `predict_proba` metric as a tool, our classifier looks pretty good, for something that we just came up with.



## Sources/References/Further Reading

[Aurelien Geron's ML Book from O'Reilly](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

[Kaggle's Public Datasets](https://www.kaggle.com/datasets)

[Portland Data Science Meetup Group](https://www.meetup.com/Portland-Data-Science-Group/?_cookie-check=gA3HOsispN8xbOgE)